In [11]:
%matplotlib inline
import sys
sys.path.append('../')
sys.path.append('../../')
sys.path.append('../../../')
sys.path.append('../../../../')
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime, timedelta
from matplotlib import pyplot as plt

from PyFin.api import *
from alphamind.api import *
from conf.models import *
from conf.config import*
from data.engines.model import Record2, Record
from alphamind.execution.naiveexecutor import NaiveExecutor
from stacking import factor_store, feature_list
from optimization.bayes_optimization_xgb import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth',100)

from data.engines.sqlengine import SQLEngine

weekly = 's4'  # s1, s2,  s3, s4, s5, s_1
record_engine = SQLEngine('sqlite:///./{}/real_tune_record_without_alpha.db'.format(weekly))


In [12]:
# ref_date_pre = '2019-12-27'

previous_pos = record_engine.fetch_record_meta(Record2, trade_date=None)
previous_pos

,trade_date,weight,industry,er,code
0,2019-12-25,0.031366,电子,0.041865,600745
1,2019-12-25,0.026310,电子,0.033859,603501
2,2019-12-25,0.025901,计算机,0.009972,600536
3,2019-12-25,0.025870,机械设备,0.014822,300450
4,2019-12-25,0.025850,医药生物,0.009488,600763
5,2019-12-25,0.025604,化工,0.010085,600426
6,2019-12-25,0.025256,通信,0.011479,300383
7,2019-12-25,0.024949,国防军工,0.007667,2465
8,2019-12-25,0.024550,非银金融,0.006599,2797
9,2019-12-25,0.024509,农林牧渔,0.012026,600298


In [13]:
previous_pos['adjust_date'] = previous_pos.trade_date
previous_pos['trade_date'] = previous_pos.trade_date + timedelta(days=1)
previous_pos

,trade_date,weight,industry,er,code,adjust_date
0,2019-12-26,0.031366,电子,0.041865,600745,2019-12-25
1,2019-12-26,0.026310,电子,0.033859,603501,2019-12-25
2,2019-12-26,0.025901,计算机,0.009972,600536,2019-12-25
3,2019-12-26,0.025870,机械设备,0.014822,300450,2019-12-25
4,2019-12-26,0.025850,医药生物,0.009488,600763,2019-12-25
5,2019-12-26,0.025604,化工,0.010085,600426,2019-12-25
6,2019-12-26,0.025256,通信,0.011479,300383,2019-12-25
7,2019-12-26,0.024949,国防军工,0.007667,2465,2019-12-25
8,2019-12-26,0.024550,非银金融,0.006599,2797,2019-12-25
9,2019-12-26,0.024509,农林牧渔,0.012026,600298,2019-12-25


In [14]:
record_engine.write_data('pos_record', previous_pos, if_exists='replace')

In [15]:
record_engine.fetch_record('pos_record')

,trade_date,weight,industry,er,code,adjust_date
0,2019-12-26,0.031366,电子,0.041865,600745,2019-12-25
1,2019-12-26,0.026310,电子,0.033859,603501,2019-12-25
2,2019-12-26,0.025901,计算机,0.009972,600536,2019-12-25
3,2019-12-26,0.025870,机械设备,0.014822,300450,2019-12-25
4,2019-12-26,0.025850,医药生物,0.009488,600763,2019-12-25
5,2019-12-26,0.025604,化工,0.010085,600426,2019-12-25
6,2019-12-26,0.025256,通信,0.011479,300383,2019-12-25
7,2019-12-26,0.024949,国防军工,0.007667,2465,2019-12-25
8,2019-12-26,0.024550,非银金融,0.006599,2797,2019-12-25
9,2019-12-26,0.024509,农林牧渔,0.012026,600298,2019-12-25
